In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Training and Test file

In [9]:
train_df = pd.read_csv("Raw DataSet/[Dataset]_Module8_Train_(Employee).csv")
test_df = pd.read_csv("Raw DataSet/[Dataset]_Module8_Test_(Employee).csv")

Split data


In [10]:
combined_df = pd.concat([train_df, test_df])
combined_df = shuffle(combined_df, random_state=42)

train_ratio = 0.7
validation_ratio = 0.1
test_ratio = 0.2
train_data, temp_data = train_test_split(combined_df, test_size=(validation_ratio + test_ratio), random_state=42)
validation_data, test_data = train_test_split(temp_data, test_size=test_ratio / (validation_ratio + test_ratio), random_state=42)

Missing Data

In [11]:
for df in [train_data, validation_data, test_data]:
    missing_cols = [col for col in df.columns if df[col].isnull().mean() > 0.4]
    df.drop(columns=missing_cols, inplace=True)

    cols_to_impute = df.columns.difference(['Age', 'Attrition_rate', 'Employee_ID', 'Hometown', 'Unit', 'Decision_skill_possess', 'Compensation_and_Benefits', 'Gender', 'Relationship_Status'])

    if len(cols_to_impute)>0:
        imp = IterativeImputer(max_iter=10, random_state=0)
        df[cols_to_impute] = imp.fit_transform(df[cols_to_impute])

    df['Age'] = df['Age'].fillna(df.groupby('Unit')['Age'].transform('median'))
    df['Gender'] = df['Gender'].map({'M': 0, 'F': 1})
    df['Relationship_Status'] = df['Relationship_Status'].map({'Married': 0, 'Single': 1})
    df = pd.get_dummies(df, columns=['Hometown', 'Unit', 'Decision_skill_possess', 'Compensation_and_Benefits'])
    df.drop(columns=['Employee_ID'], inplace=True)

In [12]:
train_data.to_csv("train.csv", index=False)
validation_data.to_csv("validation.csv", index=False)
test_data.to_csv("test.csv", index=False)